### 👉 Goal: use a frozen CLIP model to check if image ↔ text embeddings align.

We’ll just do Stage 1 → Step E1.1 right now:
👉 Goal: use a frozen CLIP model to check if image ↔ text embeddings align.

You’ll:

Load a few images + captions,

Get CLIP embeddings,

Compute cosine similarities,

See if each caption matches its image.

In [ ]:
import torch, random
from transformers import CLIPProcessor, CLIPModel
from PIL import Image
import requests
from tqdm import tqdm
import matplotlib.pyplot as plt


# reproducibility
torch.manual_seed(0)
device = "cuda" if torch.cuda.is_available() else "cpu"


In [ ]:
import matplotlib.pyplot as plt

# your images and captions
images = [
    Image.open(requests.get("https://picsum.photos/id/237/400/400", stream=True).raw),
    Image.open(requests.get("https://picsum.photos/id/1025/400/400", stream=True).raw),
    Image.open(requests.get("https://picsum.photos/id/1074/400/400", stream=True).raw)
]

captions = [
    "A black dog looking at the camera",
     "A small dog wrapped in a blanket",          # image 2 ✅
    "A close-up portrait of a lion"              # image 3 ✅
]

# plot
fig, axes = plt.subplots(1, len(images), figsize=(15, 5))

if len(images) == 1:
    axes = [axes]

for ax, img, cap in zip(axes, images, captions):
    ax.imshow(img)
    ax.axis("off")
    ax.set_title(cap, fontsize=10, wrap=True)

plt.tight_layout()
plt.show()


In [ ]:
model_name = "openai/clip-vit-base-patch16"
model = CLIPModel.from_pretrained(model_name).to(device)
processor = CLIPProcessor.from_pretrained(model_name)

# embed images
with torch.no_grad():
    inputs = processor(images=images, return_tensors="pt").to(device)
    img_emb = model.get_image_features(**inputs)
    img_emb = img_emb / img_emb.norm(dim=-1, keepdim=True)

# embed texts
with torch.no_grad():
    inputs = processor(text=captions, return_tensors="pt", padding=True).to(device)
    txt_emb = model.get_text_features(**inputs)
    txt_emb = txt_emb / txt_emb.norm(dim=-1, keepdim=True)


In [ ]:
# cosine similarity matrix
sims = txt_emb @ img_emb.T
sims_np = sims.cpu().numpy()

for i, cap in enumerate(captions):
    best_idx = torch.argmax(sims[i]).item()

    # show the image
    plt.imshow(images[best_idx])
    plt.axis("off")
    plt.title(f"True caption:\n{cap}\n\nCLIP matched image #{best_idx+1}", fontsize=10)
    plt.show()

    print(f"--- Similarities for caption {i+1} ---")
    for j, val in enumerate(sims_np[i]):
        print(f"Image {j+1}: {val:.3f}")
    print()


### Goal: check whether Whisper’s audio embeddings align with MiniLM’s text embeddings.
We’ll keep it as simple and visual as before — small dataset, frozen models, cosine similarity, clear output.

In [ ]:
import torch, torchaudio
from transformers import WhisperProcessor, WhisperModel
from sentence_transformers import SentenceTransformer
from tqdm import tqdm
import matplotlib.pyplot as plt
import requests

torch.manual_seed(0)
device = "cuda" if torch.cuda.is_available() else "cpu"


In [ ]:
import io
import requests
import torchaudio

samples = [
    {
        "url": "https://huggingface.co/datasets/Narsil/asr_dummy/resolve/main/1.flac",
        "caption": "A man is speaking calmly"
    },
    {
        "url": "https://huggingface.co/datasets/Narsil/asr_dummy/resolve/main/2.flac",
        "caption": "A person is laughing loudly"
    },
    {
        "url": "https://huggingface.co/datasets/Narsil/asr_dummy/resolve/main/3.flac",
        "caption": "A dog is barking"
    }
]

audios, captions = [], []
for s in samples:
    # ✅ download to bytes buffer, then read with torchaudio
    resp = requests.get(s["url"])
    wav_bytes = io.BytesIO(resp.content)
    wav, sr = torchaudio.load(wav_bytes)
    audios.append((wav.mean(0), sr))
    captions.append(s["caption"])

print("Loaded", len(audios), "audio clips successfully ✅")


In [ ]:
# Whisper encoder (for audio)
whisper_name = "openai/whisper-tiny"
whisper = WhisperModel.from_pretrained(whisper_name).to(device).eval()
whisper_proc = WhisperProcessor.from_pretrained(whisper_name)

# MiniLM (for text)
text_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2", device=device)


In [ ]:
a_emb = []
for wav, sr in tqdm(audios, desc="Audio → Whisper"):
    # Make sure it's a single 1D numpy array
    audio_np = wav.squeeze().numpy()
    inputs = whisper_proc([audio_np], sampling_rate=sr, return_tensors="pt").to(device)

    with torch.no_grad():
        feats = whisper.encoder(inputs.input_features).last_hidden_state.mean(dim=1)
    feats = feats / feats.norm(dim=-1, keepdim=True)
    a_emb.append(feats.cpu())
a_emb = torch.cat(a_emb, dim=0)


In [ ]:
# compute text embeddings for captions
t_emb = torch.tensor(
    text_model.encode(captions, convert_to_numpy=True, normalize_embeddings=True)
)

In [ ]:
sims = (a_emb @ t_emb.T).cpu().numpy()

for i, cap in enumerate(captions):
    best = sims[i].argmax()
    print(f"\nCaption {i+1}: {cap}")
    print(f"Best-matched audio index: {best+1}")
    print("Similarity scores:", sims[i].round(3))

    # Simple bar plot
    plt.bar(range(1, len(audios)+1), sims[i])
    plt.title(f"Audio {i+1} vs Texts")
    plt.xlabel("Text index"); plt.ylabel("Cosine similarity")
    plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity

# ensure everything is numpy
a_np = a_emb.numpy()
t_np = t_emb.numpy()
i_np = img_emb.cpu().numpy()

In [ ]:
a_np.shape

In [ ]:
from sklearn.decomposition import PCA
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

common_dim = min(2, t_np.shape[0], i_np.shape[0])  # safe value
pca_t = PCA(n_components=common_dim).fit(t_np)
pca_i = PCA(n_components=common_dim).fit(i_np)

t_proj = pca_t.transform(t_np)
i_proj = pca_i.transform(i_np)

sim_t_i = cosine_similarity(t_proj, i_proj)


In [ ]:
# 1️⃣ Audio → Text similarity
sim_a_t = cosine_similarity(a_np, t_np)

In [ ]:
# pick the best-matching text for each audio
best_text_idx = np.argmax(sim_a_t, axis=1)


In [ ]:
# # 2️⃣ Text → Image similarity (CLIP space)
# sim_t_i = cosine_similarity(t_np, i_np)
sim_a_i = sim_a_t[np.arange(len(best_text_idx)), best_text_idx][:, None] * sim_t_i[best_text_idx]
best_img_idx = np.argmax(sim_a_i, axis=1)


In [ ]:
# compute final Audio → Image score through the text pivot
sim_a_i = sim_a_t[np.arange(len(best_text_idx)), best_text_idx][:, None] * sim_t_i[best_text_idx]

# for each audio, get the image with the highest combined similarity
best_img_idx = np.argmax(sim_a_i, axis=1)


In [ ]:
for n, (audio_idx, img_idx) in enumerate(zip(range(len(best_img_idx)), best_img_idx)):
    print(f"\nAudio {audio_idx+1} predicted best image: {img_idx+1}")
    print("Matched caption:", captions[best_text_idx[audio_idx]])

    plt.imshow(images[img_idx])
    plt.axis("off")
    plt.title(f"Audio {audio_idx+1} → Image {img_idx+1}\nvia text '{captions[best_text_idx[audio_idx]]}'")
    plt.show()


# Title: “Zero-shot Image–Text Retrieval (CLIP-ViT-B/16)”

Objective: Show that a frozen vision-language model already aligns images ↔ captions.

Setup: CLIP ViT-B/16, COCO-2017 val, N=12 pairs, no training.

Procedure: Encode images & captions → cosine similarity → retrieval.

Metrics: Acc@1 (caption→image), similarity heatmap, top-3 qualitative results.

Takeaway: CLIP gets strong text↔image alignment out-of-the-box → a solid baseline to compare against.

In [ ]:
# ! pip install seaborn

In [ ]:
from datasets import load_dataset
from itertools import islice

# streaming = True lets us pull just a few samples quickly
ds = load_dataset("allenai/pixmo-cap", split="train", streaming=True)

# take 2 samples only
samples = list(islice(ds, 2))

print(f"Loaded {len(samples)} examples")
for i, s in enumerate(samples):
    print(f"\nSample {i+1} caption:\n{s['caption'][:200]} ...")


In [ ]:
import requests
from PIL import Image
import matplotlib.pyplot as plt

fig, axes = plt.subplots(1, len(samples), figsize=(14, 5))
if len(samples) == 1:
    axes = [axes]

for ax, s in zip(axes, samples):
    img = Image.open(requests.get(s["image_url"], stream=True).raw).convert("RGB")
    ax.imshow(img)
    ax.axis("off")
    ax.set_title(s["caption"][:120] + "…", fontsize=10, wrap=True)
    break
plt.tight_layout()
plt.show()


In [ ]:
import torch, numpy as np, matplotlib.pyplot as plt, seaborn as sns, requests
from PIL import Image
from datasets import load_dataset
from itertools import islice
from transformers import AutoModel, AutoProcessor
from tqdm import tqdm

In [ ]:
pairs = []
for ex in islice(ds, 60):  # overfetch a bit to skip any bad rows
    cap = ex.get("caption", None)
    url = ex.get("image_url", None)
    if not cap or not url:
        continue
    try:
        img = Image.open(requests.get(url, stream=True).raw).convert("RGB")
    except Exception:
        continue
    pairs.append((img, cap))
    if len(pairs) == N:
        break

images = [im for im, _ in pairs]
captions = [tx for _, tx in pairs]
print(f"Loaded {len(images)} PixMo samples")


In [ ]:

# ---------- SigLIP model ----------
model_id = "google/siglip-base-patch16-224"
model = AutoModel.from_pretrained(model_id).to(device).eval()         # returns SiglipModel
proc  = AutoProcessor.from_pretrained(model_id)                       # returns SiglipProcessor


In [ ]:


# ---------- Embeddings (normalized) ----------
with torch.no_grad():
    img_emb = model.get_image_features(**proc(images=images, return_tensors="pt").to(device))
    img_emb = img_emb / img_emb.norm(dim=-1, keepdim=True)
    txt_emb = model.get_text_features(**proc(text=captions, return_tensors="pt", padding=True, truncation=True).to(device))
    txt_emb = txt_emb / txt_emb.norm(dim=-1, keepdim=True)

# ---------- Similarity & Acc@1 ----------
S = (txt_emb @ img_emb.T).detach().cpu().numpy()   # [N_text, N_img]
pred = S.argmax(axis=1)
acc1 = (pred == np.arange(len(captions))).mean()
print(f"Acc@1 (caption→image): {acc1:.2f}")

# ---------- Figure A: Heatmap ----------
plt.figure(figsize=(6, 4.5))
sns.heatmap(S, annot=False, cmap="viridis")
plt.xlabel("image index"); plt.ylabel("caption index")
plt.title(f"SigLIP cosine similarity (Acc@1={acc1:.2f})")
plt.tight_layout()
plt.savefig("siglip_heatmap.png", dpi=200)
plt.show()

# ---------- Figure B: Top-3 retrieval gallery ----------
def show_topk_gallery(k_ids=(0,1,2), topk=3):
    for cid in k_ids:
        order = np.argsort(-S[cid])[:topk]
        fig, axes = plt.subplots(1, topk, figsize=(10, 3.2))
        for j, ax in enumerate(axes):
            ax.imshow(images[order[j]]); ax.axis("off")
            ax.set_title(f"rank {j+1}  s={S[cid, order[j]]:.2f}", fontsize=9)
        fig.suptitle(f"Query caption:\n{captions[cid]}", fontsize=11)
        plt.tight_layout()
        plt.savefig(f"siglip_top{topk}_caption{cid}.png", dpi=200)
        plt.show()

show_topk_gallery(k_ids=tuple(range(min(3, len(captions)))), topk=3)
